In [3]:
import numpy as np

def soft_threshold(x, lam):
    """Soft-thresholding operator S(x, lam) = sign(x)*max(|x|-lam, 0)."""
    return np.sign(x) * np.maximum(np.abs(x) - lam, 0.0)

def lasso_coordinate_descent(X, y, lam=1.0, max_iter=1000, tol=1e-6):
    """
    Solve Lasso: 1/2 * ||y - Xbeta||^2 + lam * ||beta||_1
    using cyclical coordinate descent.
    """
    n, p = X.shape

    # Precompute
    V = X.T @ X  # p x p
    s = X.T @ y  # p x 1

    beta = np.zeros(p)  # initialize coefficients

    for _ in range(max_iter):
        beta_old = beta.copy()

        for j in range(p):
            # partial residual: s_j - sum_{k != j} V_{kj} * beta_k
            r_j = s[j] - np.dot(V[j, :], beta) + V[j, j]*beta[j]

            # Update beta_j
            beta[j] = soft_threshold(r_j, lam) / (V[j, j] + 1e-12)  # add small epsilon to avoid /0

        # check for convergence
        if np.linalg.norm(beta - beta_old) < tol:
            break

    return beta


# --- Validation on simulated data ---
if __name__ == "__main__":
    np.random.seed(123)

    # Generate simulated data
    n, p = 200, 10
    X = np.random.randn(n, p)
    # True beta with only a few non-zeros
    beta_true = np.array([5, -3, 0, 0, 2, 0, 0, 0, 0, 0], dtype=float)
    # Response with noise
    y = X @ beta_true + 0.5 * np.random.randn(n)

    lam = 1.0
    beta_est = lasso_coordinate_descent(X, y, lam=lam, max_iter=1000, tol=1e-8)
    print("True beta:", beta_true)
    print("Estimated beta:", beta_est)


True beta: [ 5. -3.  0.  0.  2.  0.  0.  0.  0.  0.]
Estimated beta: [ 5.01651435e+00 -2.95409981e+00  0.00000000e+00 -2.35553764e-02
  1.99454076e+00  4.49970095e-04  4.32995799e-03 -2.45489737e-02
 -1.20736754e-02 -1.35658189e-02]


In [4]:
import numpy as np

def soft_threshold_vector(z, lam):
    """Apply soft-thresholding to a vector z with threshold lam."""
    return np.sign(z) * np.maximum(np.abs(z) - lam, 0)

def graphical_lasso(S, lam=0.1, max_iter=100, tol=1e-4):
    """
    Graphical Lasso using the block-coordinate descent (Friedman et al. style).

    S: empirical covariance matrix (p x p)
    lam: regularization parameter
    Returns: (Theta, Sigma) the estimated precision and covariance
    """
    p = S.shape[0]
    # Initialize Theta (precision) -- e.g. with the diagonal of S or identity
    # A typical choice: invert (S + lam * I) to have a strictly positive definite
    W = np.linalg.inv(S + lam * np.eye(p))
    Theta = W.copy()

    for _ in range(max_iter):
        Theta_old = Theta.copy()

        for j in range(p):
            # Partition indices
            not_j = np.arange(p) != j

            S11 = S[not_j][:, not_j]
            s12 = S[not_j, j]

            # Theta11 => block (not_j, not_j)
            # We extract the sub-block we will modify
            T11 = Theta[not_j][:, not_j]
            t12 = Theta[not_j, j]

            # Solve for new t12 using Lasso-like subproblem:
            #   argmin_b  b^T T11 b - 2 s12^T b + lam * ||b||_1


            # We define: W_11 = T11^{-1} (the sub‐cov)
            W_11 = np.linalg.inv(T11)
            gamma = - W_11 @ s12  # “pseudo” target (see derivations in Friedman (2008))

            # Now solve the Lasso problem for b:
            #   argmin_b  1/2 * b^T W_11^-1 b + ...
            # but let's do a simpler approach: coordinate descent in practice.

            # Initialize b with current t12
            b = t12.copy()

            # Simple coordinate descent for the subproblem
            for it_inner in range(50):
                b_old = b.copy()
                for i_idx in range(len(b)):
                    r_i = gamma[i_idx] - (W_11[i_idx, :] @ b) + W_11[i_idx, i_idx] * b[i_idx]
                    # soft threshold
                    b[i_idx] = soft_threshold(r_i, lam) / W_11[i_idx, i_idx]

                if np.linalg.norm(b - b_old) < 1e-8:
                    break

            # Update t12 in Theta
            t12_new = b

            # Next, we need to update Theta_{jj} accordingly
            #formula: Theta_{jj} = 1 / ( w_{jj} - w_{j,-j} * b )
            # Then we set Theta_jj to the scalar:

            # Compute the value for Theta_{j,j}
            denom = S[j, j] - (s12.T @ t12_new)
            Theta_jj_new = 1.0 / max(denom, 1e-12)

            # Put the updates back
            Theta[not_j, j] = t12_new
            Theta[j, not_j] = t12_new
            Theta[j, j] = Theta_jj_new

        # check for convergence
        if np.linalg.norm(Theta - Theta_old, ord='fro') < tol:
            break

    # Symmetrize just in case of numerical issues
    Theta = 0.5 * (Theta + Theta.T)

    # Invert Theta to get the covariance
    Sigma_est = np.linalg.inv(Theta)

    return Theta, Sigma_est


# --- Validation using simulated data ---
if __name__ == "__main__":
    np.random.seed(42)
    p = 5
    n = 500

    # Generate a random precision matrix Theta_true with some sparse off-diagonal
    # e.g., a random diagonal + a few random off-diagonals
    A = 0.6 * np.random.randn(p, p)
    # Make it symmetric
    A = 0.5*(A + A.T)
    # Shift the diagonal to ensure positive definite
    diag_add = p * np.eye(p)
    Theta_true = A + diag_add
    Theta_true = 0.5 * (Theta_true + Theta_true.T)  # symmetrize

    # Check positive definiteness (do a shift if needed)
    # We'll assume it's PD enough. Or use e.g. a nearPD approach if needed.

    # Generate data X ~ N(0, Sigma_true) where Sigma_true = Theta_true^{-1}
    Sigma_true = np.linalg.inv(Theta_true)
    X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma_true, size=n)

    # Empirical covariance
    S = np.cov(X, rowvar=False)

    # Run Graphical Lasso
    lam = 0.1
    Theta_est, Sigma_est = graphical_lasso(S, lam=lam, max_iter=100, tol=1e-4)

    print("True Precision:\n", Theta_true)
    print("Estimated Precision:\n", Theta_est)

    # Check special cases:
    # 1) lambda -> 0: we should get close to the MLE, i.e. S^{-1}.
    # 2) lambda -> large: we expect more shrinkage, eventually diagonal.


True Precision:
 [[ 5.29802849 -0.11172038  0.05528125  0.2882227   0.36944862]
 [-0.11172038  5.94752769  0.09051149 -0.44469165  0.09503512]
 [ 0.05528125  0.09051149  5.14517736 -0.47970987 -0.49721689]
 [ 0.2882227  -0.44469165 -0.47970987  4.45518555 -0.85111557]
 [ 0.36944862  0.09503512 -0.49721689 -0.85111557  4.67337037]]
Estimated Precision:
 [[ 5.66514038  0.          0.          0.          0.        ]
 [ 0.          5.64671335  0.         -0.          0.        ]
 [ 0.          0.          5.77929504 -0.         -0.        ]
 [ 0.         -0.         -0.          4.14891882 -0.        ]
 [ 0.          0.         -0.         -0.          4.43464563]]
